In [1]:
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from zipfile import ZipFile
import edgar_utils as eu
from edgar_utils import lookup_region
from edgar_utils import Filing
from bs4 import BeautifulSoup
from collections import defaultdict
from collections import Counter

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#q1

with ZipFile('server_log.zip', 'r') as zf:
    with zf.open('rows.csv') as file:
        df = pd.read_csv(file)
        
total_size = int(df['size'].sum())
total_size

24801002666

In [4]:
#q2

# df['count'] = 1
# required_df = df[['ip', 'count']]
# sum_df = required_df.groupby('ip').sum()
# dict_needed = sum_df['count'].sort_values(ascending = False).head(10).to_dict()
# dict_needed

dict_needed = df["ip"].value_counts().head(10).to_dict()
dict_needed

{'54.152.17.ccg': 12562,
 '183.195.251.hah': 6524,
 '52.45.218.ihf': 5562,
 '68.180.231.abf': 5493,
 '204.212.175.bch': 4708,
 '103.238.106.gif': 4428,
 '208.77.215.jeh': 3903,
 '208.77.214.jeh': 3806,
 '217.174.255.dgd': 3551,
 '82.13.163.caf': 3527}

In [5]:
#q3

# num_requests = len(df)

# error_counts = 0

# for code in df['code']:
#     if code >= 400.0:
#         error_counts += 1
        
# error_counts / num_requests

num_requests = len(df)

error_counts = df[df['code'] >= 400.0].shape[0]

error_counts / num_requests

0.03466852724527611

In [6]:
#q4

# new_df = df[['cik', 'accession', 'extention']]
# sum_df = new_df.groupby(['cik', 'accession', 'extention']).size().sort_values(ascending = False).index.to_list()
# cik = int(sum_df[1][0])
# accession = sum_df[1][1]
# extention = sum_df[1][2]
# string_needed = f"{cik}/{accession}/{extention}"
# string_needed

df['file_path'] = df["cik"].astype(int).astype(str) + "/" + df["accession"] + "/" + df["extention"]
file_counts = df["file_path"].value_counts()
file_counts.index[1]

'1584509/0001584509-16-000514/armk-20160930_def.xml'

In [7]:
#q5

# unique_ips = df['ip'].unique()
# ip_to_region = {ip: eu.lookup_region(ip) for ip in unique_ips}

# df['region'] = df['ip'].map(ip_to_region)

# # Step 3: Group by region and sum the counts to find the region with max resources
# max_resources_region = df.groupby('region')['count'].sum().idxmax()
# max_resources_region

df['region'] = df['ip'].apply(lookup_region)
region_counts = df['region'].value_counts()
region_counts.idxmax()

'United States of America'

In [8]:
#q6

high_volume_ip_counts = df['ip'].value_counts()
high_volume_ips = high_volume_ip_counts[high_volume_ip_counts > 300].index

df['is_high_volume'] = df['ip'].isin(high_volume_ips)

region_ip_counts = df.groupby('region')['ip'].nunique()  # Total unique IPs in each region
high_volume_region_counts = df[df['is_high_volume']].groupby('region')['ip'].nunique()  # High-volume IPs in each region

high_volume_fraction = (high_volume_region_counts / region_ip_counts).fillna(0).to_dict()
high_volume_fraction

{'-': 0.0,
 'Algeria': 0.0,
 'Angola': 0.0,
 'Armenia': 0.0,
 'Australia': 0.07692307692307693,
 'Austria': 0.0,
 'Bangladesh': 0.0,
 'Brazil': 0.0,
 'Bulgaria': 0.3333333333333333,
 'Canada': 0.05357142857142857,
 'China': 0.016018306636155607,
 'Croatia': 0.0,
 'Cyprus': 0.0,
 'Czechia': 0.3333333333333333,
 'Denmark': 0.0,
 'Egypt': 0.0,
 'Estonia': 0.0,
 'Finland': 0.0,
 'France': 0.0036900369003690036,
 'Germany': 0.047619047619047616,
 'Greece': 0.0,
 'Honduras': 0.0,
 'Hong Kong': 0.037037037037037035,
 'Hungary': 0.0,
 'India': 0.08823529411764706,
 'Indonesia': 0.0,
 'Iraq': 0.0,
 'Ireland': 0.625,
 'Israel': 0.0,
 'Italy': 0.0,
 'Japan': 0.0,
 'Jersey': 0.0,
 'Jordan': 0.0,
 'Korea (Republic of)': 0.2,
 'Latvia': 0.0,
 'Lebanon': 0.0,
 'Liberia': 0.0,
 'Luxembourg': 0.0,
 'Malaysia': 0.0,
 'Mexico': 0.0,
 'Mongolia': 0.0,
 'Morocco': 0.0,
 'Netherlands': 0.0,
 'Nigeria': 0.0,
 'North Macedonia': 0.0,
 'Pakistan': 0.0,
 'Philippines': 0.0,
 'Poland': 0.0,
 'Portugal': 0.0,
 'R

In [9]:
#q7

file_path = "886982/0000769993-16-001958/-index.htm"

with ZipFile("docs.zip", "r") as zf:
    with zf.open(file_path) as file:
        html_content = file.read().decode('utf-8')
        
filing_obj = Filing(html_content)
filing_obj.dates

['2016-12-30', '2016-12-30', '2016-12-20']

In [10]:
filings = {}

with ZipFile('docs.zip', "r") as zf:
    for file_path in zf.namelist():
        if file_path.endswith(".htm") or file_path.endswith(".html"):
            with zf.open(file_path) as file:
                html_content = file.read().decode("utf-8")
                filing_obj = Filing(html_content)
                filings[file_path] = filing_obj

In [11]:
filings['850693/0000850693-07-000159/-index.htm'].addresses

['P.O. BOX 19534\nIRVINE CA 92713-9534',
 '2525 DUPONT DRIVE\nIRVINE CA 92612\n7142464500',
 '2525 DUPONT DR\nIRVINE CA 92612']

In [12]:
#q8

state_distribution = defaultdict(int)

for filing_obj in filings.values():
    state = filing_obj.state()
    state_distribution[state] += 1
    
state_distribution = dict(state_distribution)
state_distribution

{'CA': 92,
 'NJ': 23,
 'MS': 1,
 'MA': 30,
 'CO': 25,
 'NY': 83,
 None: 56,
 'TX': 67,
 'FL': 21,
 'AL': 1,
 'IN': 5,
 'CT': 14,
 'UT': 5,
 'MD': 13,
 'KS': 5,
 'VA': 15,
 'OH': 10,
 'SD': 1,
 'DE': 9,
 'IL': 25,
 'GA': 9,
 'NC': 9,
 'PA': 25,
 'TN': 4,
 'MN': 15,
 'NM': 1,
 'KY': 2,
 'OK': 7,
 'AR': 1,
 'NV': 6,
 'VT': 1,
 'MO': 4,
 'MI': 11,
 'NE': 2,
 'IA': 6,
 'AZ': 5,
 'WI': 9,
 'LA': 2,
 'ID': 1,
 'OR': 2,
 'WV': 2,
 'WA': 3,
 'DC': 1,
 'ME': 1}

In [13]:
#q9

all_addresses = []

for filing_obj in filings.values():
    all_addresses.extend(filing_obj.addresses)
    
address_distribution = Counter(all_addresses)
top_10_addresses = address_distribution.most_common(10)
top_10_addresses_dict = dict(top_10_addresses)
top_10_addresses_dict

{'2000 AVENUE OF THE STARS, 12TH FLOOR\nLOS ANGELES CA 90067': 134,
 '2000 AVENUE OF THE STARS, 12TH FLOOR\nLOS ANGELES CA 90067\n3102014100': 113,
 '3 LANDMARK SQUARE\nSUITE 500\nSTAMFORD CT 06901\n2033564400': 60,
 'C/O KKR ASSET MANAGEMENT LLC\n555 CALIFORNIA STREET, 50TH FLOOR\nSAN FRANCISCO CA 94104': 36,
 'C/O ARES MANAGEMENT LLC\n2000 AVENUE OF THE STARS, 12TH FLOOR\nLOS ANGELES CA 90067': 35,
 '4740 AGAR DRIVE\nRICHMOND A1 V7B 1A3': 25,
 'CENTRALIS S.A., 8-10 AVENUE DE LA GARE\nLUXEMBOURG N4 L-1610': 25,
 'CENTRALIS S.A., 8-10 AVENUE DE LA GARE\nLUXEMBOURG N4 L-1610\n352-26-186-1': 25,
 '3 LANDMARK SQUARE\nSUITE 500\nSTAMFORD CT 06901': 24,
 '801 CHERRY STREET\nSUITE 2100\nFORT WORTH TX 76102': 22}